# Dog Brid Identification
Kaggle competetion https://www.kaggle.com/c/dog-breed-identification

## Let's first import the required librariees

In [1]:
import pandas
from os.path import join

from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

from keras.initializers import RandomNormal
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation, Dense, Dropout, Flatten
from keras.layers.pooling import MaxPooling2D

from keras.models import Sequential
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

from datagen import DataGenerator

Using TensorFlow backend.


Define few paths

In [2]:
# Define few paths
data_dir = './data'
train_data_dir = './data/train'
test_data_dir = './data/test'

Load the label file

In [3]:
# Read the labels file
label_file = pandas.read_csv(join(data_dir, 'labels.csv'), nrows=3200)

In [4]:
print(label_file.head(5))

                                 id             breed
0  000bec180eb18c7604dcecc8fe0dba07       boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97             dingo
2  001cdf01b096e06d78e9e5112d419397          pekinese
3  00214f311d5d2247d5dfe4fe24b2303d          bluetick
4  0021f9ceb3235effd7fcde7f7538ed62  golden_retriever


Initialize few inportant variables

In [5]:
# Initialize the number of classes
n_classes = len(label_file['breed'].unique())
# Total number of samples present, train + val
n_samples = len(label_file)

Create important dictionaries

In [6]:
# create a label to idx dictionary. Its basically string label to int label dictionary.
label_to_idx = {}
# dictionary to map int label to string label
idx_to_label = {}
unique_labels = label_file['breed'].unique()
for i in range(0, n_classes):
    label_to_idx[unique_labels[i]] = i
    idx_to_label[i] = unique_labels[i]

Let's create the datasets

In [7]:
# Data sets
val_percentage = 0.1
train, val = train_test_split(label_file, test_size=val_percentage)

# this dictionary lists all the ids in train and val set
partition = {'train': train['id'].tolist(), 'val': val['id'].tolist()}
# this is a id to label dictionary
id_to_labels = {}
labels_oh = {}  # id to one-hot label dictionary
for i in range(0, n_samples):
    l_id, lb = label_file.iloc[i]  # id, string label
    id_to_labels[l_id] = lb
    # add the integer value of labels in the dataframe itself, this will help to create one-hot represetation
    labels_oh[l_id] = label_to_idx[lb]

Create and initialize the model parameters

In [8]:
# Parameters
dim_x = 224
dim_y = 224
batch_size = 32
n_channels = 3
params = {'dim_x': dim_x,
          'dim_y': dim_y,
          'batch_size': 320,
          'n_classes': n_classes,
          'shuffle': True}

To use with ImageGenerator we must have some statistical measures about the data. The `get_sample()` function reads `n` samples to be used  by ImageGenerator.

In [9]:
def get_sample(num, size):
    sample = []
    # print(partition['train'])
    # print(partition['val'])
    # print(labels_oh)
    for i in range(0, num):
        img_id = partition['train'][i]
        # print("i=%s, id=%s" % (i, img_id))
        # print(partition['train'])
        img = image.load_img(join(data_dir, 'train', '%s.jpg' % img_id), target_size=size)
        img = image.img_to_array(img)
        sample.append(img)
    return sample

## Design the model

In [13]:
def get_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(dim_x, dim_y, n_channels))

    '''
    # #------------------------------ Model ---------------------------------------# #

    # Design model
    model = Sequential()

    # Add layers
    model.add(Conv2D(filters=32,
                     kernel_size=9,
                     strides=2,
                     padding='valid',
                     kernel_initializer=RandomNormal(mean=0.0, stddev=0.05),
                     bias_initializer='zeros',
                     kernel_regularizer=l2(0.001),
                     bias_regularizer=None,
                     input_shape=(dim_x, dim_y, n_channels)))

    model.add(LeakyReLU(alpha=0.1))

    model.add(Dropout(rate=0.2))

    model.add(Conv2D(filters=64,
                     kernel_size=5,
                     strides=1,
                     padding='valid',
                     kernel_initializer=RandomNormal(mean=0.0, stddev=0.05),
                     bias_initializer='zeros',
                     kernel_regularizer=l2(0.001),
                     bias_regularizer=None))

    model.add(LeakyReLU(alpha=0.1))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(rate=0.3))

    model.add(Flatten())

    model.add(Dense(units=128,
                    kernel_initializer=RandomNormal(mean=0.0, stddev=0.01),
                    bias_initializer='zeros',
                    kernel_regularizer=l2(0.001),
                    bias_regularizer=None))

    model.add(Dropout(rate=0.4))
    '''

    x = base_model.output
    x = Flatten()(x)
    x = Dense(units=n_classes,
              kernel_initializer=RandomNormal(mean=0.0, stddev=0.01),
              bias_initializer='zeros',
              kernel_regularizer=l2(0.001),
              bias_regularizer=None)(x)

    predictions = Activation('softmax')(x)

    # create graph of new model
    model = Model(inputs=base_model.input, outputs=predictions)

    # freeze all convolutional base model layers
    for layer in base_model.layers:
        layer.trainable = False

    return model

## Load model weights and compile.

In [14]:
# get the model
model = get_model()
# define the checkpoint
file_path = "best_model.h5"
checkpoint = ModelCheckpoint(file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# compile the model
model.compile(optimizer=Adam(lr=0.01, decay=0.00016667),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

94658560/94653016 [==============================] - 33s 0us/step


Create the data generators.

In [15]:
# Generators
training_generator = DataGenerator(**params).generate(labels_oh, partition['train'], n_classes)
validation_generator = DataGenerator(**params).generate(labels_oh, partition['val'], n_classes)

# #------------------------------ Data Generator ---------------------------------------# #

datagen = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

# let's say X_sample is a small-ish but statistically representative sample of your data
X_sample = get_sample(5, (dim_x, dim_y))
datagen.fit(X_sample)

Now, train the model!

In [16]:

# let's say you have an ImageNet generator that yields ~10k samples at a time.
nb_epoch = 5
for e in range(nb_epoch):
    print("epoch %d" % e)
    g = 1
    for X_train, Y_train in DataGenerator(**params).generate(labels_oh, partition['train'], n_classes):  # these are chunks of ~10k pictures
        print("<---------- Gen level = %s ----------->" % g)
        f = 1
        for X_batch, Y_batch in datagen.flow(X_train, Y_train, batch_size=batch_size):  # these are chunks of 32 samples
            print("Flow level = %s" % f)
            # print("X_batch = %s, Y_batch = %s" % (X_batch.shape, Y_batch.shape))
            loss = model.fit(X_batch, Y_batch,
                             steps_per_epoch=int(len(X_train) / batch_size),
                             callbacks=callbacks_list,
                             verbose=2)
            if f == int(len(X_train) / len(X_batch)):
                break
            f += 1
        if g == int(len(partition['train']) / len(X_train)):
            break
        g += 1

'''
# Train model on dataset
model.fit_generator(generator=training_generator,
                    steps_per_epoch=len(partition['train']) / batch_size,
                    validation_data=validation_generator,
                    verbose=2,
                    epochs=5,
                    # validation_steps=2
                    validation_steps=len(partition['val'])/batch_size
                    )
'''

epoch 0
<---------- Gen level = 1 ----------->
Flow level = 1
Epoch 1/1
Epoch 00001: loss improved from inf to 5.24723, saving model to best_model.h5
 - 9s - loss: 5.2472 - acc: 0.0375
Flow level = 2
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 12.5442 - acc: 0.0312
Flow level = 3
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 14.7197 - acc: 0.0187
Flow level = 4
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 16.0984 - acc: 0.0063
Flow level = 5
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 16.2807 - acc: 0.0000e+00
Flow level = 6
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 16.2504 - acc: 0.0000e+00
Flow level = 7
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 16.2336 - acc: 0.0000e+00
Flow level = 8
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 16.2226 - acc: 0.0000e+00
Flow level = 9
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 16.2141 - acc: 0.0000e+00
Flow level = 10
Epoch 1/1
Epoch 000

Flow level = 1
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 16.1200 - acc: 0.0000e+00
Flow level = 2
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 14.0925 - acc: 0.0125
Flow level = 3
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 14.8761 - acc: 0.0438
Flow level = 4
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 17.6386 - acc: 0.0000e+00
Flow level = 5
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 17.5795 - acc: 0.0000e+00
Flow level = 6
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 17.2605 - acc: 0.0000e+00
Flow level = 7
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 16.2436 - acc: 0.0312
Flow level = 8
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 16.7743 - acc: 0.0000e+00
Flow level = 9
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 16.6430 - acc: 0.0000e+00
Flow level = 10
Epoch 1/1
Epoch 00001: loss did not improve
 - 2s - loss: 16.5478 - acc: 0.0000e+00
epoch 1
<---------- G

KeyboardInterrupt: 